## Handwerkliche Übungsaufgaben zu den Datenbank-Operationen in Pandas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
aisles = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/departments.csv')
orders = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/products.csv')

In [ ]:
orders.head(n=20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [ ]:
orders.count

<bound method DataFrame.count of          order_id  user_id  ... order_hour_of_day  days_since_prior_order
0         2539329        1  ...                 8                     NaN
1         2398795        1  ...                 7                    15.0
2          473747        1  ...                12                    21.0
3         2254736        1  ...                 7                    29.0
4          431534        1  ...                15                    28.0
...           ...      ...  ...               ...                     ...
3421078   2266710   206209  ...                18                    29.0
3421079   1854736   206209  ...                10                    30.0
3421080    626363   206209  ...                12                    18.0
3421081   2977660   206209  ...                12                     7.0
3421082    272231   206209  ...                14                    30.0

[3421083 rows x 7 columns]>

# Aufgabe 1:
Zum 'orders' DataFrame soll eine Spalte 'factor' hinzugefügt werden, die für jede Bestellung jeweils einen Wert d/dm enthält, wobei d 'days_since_prior_order' der betreffenden Bestellung ist, und dm der Durchschnitt der 'days_since_prior_order' aller Bestellungen des jeweiligen Users. Mit anderen Worten: Der Wert der neuen 'orders'-Spalte soll anzeigen, welche Bestellungen eines Users gemessen am üblichen Verhalten dieses Users schnell nach der vorhergehenden Bestellung erfolgt sind. 
### Zeigen Sie als Ergebnis die gleichen 20 ersten Zeilen des 'oders'DataFrame wie oben, aber ergänzu um die neue 'factors'-Spalte. 

In [ ]:
user_attribs = orders[['user_id', 'days_since_prior_order']].groupby(['user_id']).mean()

In [ ]:
user_attribs.rename({'days_since_prior_order':'avg_days_since_prior_order'}, axis='columns', inplace=True)
user_attribs.head()

,avg_days_since_prior_order
user_id,
1,19.000000
2,16.285714
3,12.000000
4,17.000000
5,11.500000


In [ ]:
# Formal stuff to make 'user_id' a column
user_attribs.reset_index(inplace=True)
user_attribs.head()

,user_id,avg_days_since_prior_order
0,1,19.000000
1,2,16.285714
2,3,12.000000
3,4,17.000000
4,5,11.500000


In [ ]:
# Join on userID tables 'orders' to 'user_attribs'
h = pd.merge(orders[['order_id', 'user_id', 'days_since_prior_order']], user_attribs)

In [ ]:
# add new column 'factor' which for each order specifies by how much
# the 'days_since_prior_order' are above/below this users avg
h['factor'] = h['days_since_prior_order']/h['avg_days_since_prior_order']

In [ ]:
h.head()

,order_id,user_id,days_since_prior_order,avg_days_since_prior_order,factor
0,2539329,1,NaN,19.0,NaN
1,2398795,1,15.0,19.0,0.789474
2,473747,1,21.0,19.0,1.105263
3,2254736,1,29.0,19.0,1.526316
4,431534,1,28.0,19.0,1.473684


In [ ]:
h.factor.sum(skipna=False)

nan

In [ ]:
h.factor.isnull().sum()

206232

In [ ]:
len(h.factor)

3421083

# Aufgabe 2:
Erzeuegen Sie und lassen Sie ausgeben eine Tabelle, die für jeden der 7 Wochentage anzeigt, welcher Anteil der Bestellungen 'organic bananas' enthält

In [ ]:
# Liste der Produkt-IDs der relevanten Produkte ermitteln
products[products.product_name.str.match('.*Organic Banana.*', case=False)]

,product_id,product_name,aisle_id,department_id
2476,2477,Organic Bananas Stage 2 Baby Food,92,18
3878,3879,Organic Bananas Beet & Blueberry Baby Food,92,18
6891,6892,Organic Banana Vanilla Smooth And Creamy Low F...,120,16
12617,12618,Organic Bananas Bananas Bananas Super Smooth P...,92,18
13175,13176,Bag of Organic Bananas,24,4
14228,14229,"Soft Baked Oat Bars, Organic Banana Carrots",92,18
20562,20563,1F Organic Bananas Organic Purees Fruit,92,18
25019,25020,Organic Banana Chips,50,19
25370,25371,Organic Banana Blueberry Baby Food Puree,92,18
29318,29319,Organic Banana Baby Breakfast (from 6 Months),92,18


In [ ]:
# enge Auslegung der Aufgabe. Benutze nur "Bag od Organic Bananas" und "Organic Banana" 
products_filter = products[products['product_name'].isin(["Bag of Organic Bananas","Organic Banana"])]
products_filter
products_filter = products_filter['product_id']
products_filter

13175    13176
37066    37067
Name: product_id, dtype: int64

In [ ]:
# Füge Spalte "contains_bananas" zum DF "orders" hinzufügen

# 1. order_ids der betreffenden Bestellungen ermitteln:
order_products = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/order_products__prior.csv', usecols=['order_id', 'product_id', 'reordered'])
order_products_train = pd.read_csv('/content/drive/MyDrive/DS/Uebung 03 (Grouping and Aggregation)/order_products__train.csv', usecols=['order_id', 'product_id', 'reordered'])
order_products.append(order_products_train)
order_products_with_bananas = order_products[order_products['product_id'].isin(products_filter)]
order_products_with_bananas.head()

,order_id,product_id,reordered
30,5,13176,1
241,27,13176,1
286,29,13176,1
304,32,13176,1
341,42,13176,1


In [ ]:
# IDs of all orders with bananas
orders_with_bananas = order_products_with_bananas['order_id'].unique()
orders_with_bananas[:10]

array([ 5, 27, 29, 32, 42, 55, 58, 60, 67, 83])

In [ ]:
# set column contains_bananas 0 for all rows
orders['contains_bananas'] = 0

In [ ]:
# locate all orders, where order_id isin the list of order_products_with_bananas
orders.loc[orders['order_id'].isin(orders_with_bananas), 'contains_bananas'] = 1

In [ ]:
# Für jeden Wochentag: Welcher Anteil der Bestellungen enthält Bananen, 
# d.h. z.B. Anzahl an montäglichen Bestellungen, die Bananen enthalten, durch Anzahl an montäglichen Bestellungen insgesamt. 
# ACHTUNG! 
# Das ist etwas anderes als "welcher Anteil aller Bananenbestellungen erfolgt an einem Montag" - das wäre
# Anzahl an montäglichen Bestellungen, die Bananen enthalten, durch Anzahl aller Bestellungen, die Bananen enthalten.  
# Das war nicht gefragt

In [ ]:
# Gruppieren nach Wochentag
orders[['order_dow', 'contains_bananas']].groupby(['order_dow']).mean()

,contains_bananas
order_dow,
0,0.124823
1,0.124020
2,0.112569
3,0.109227
4,0.109908
5,0.112571
6,0.115396


# Aufgabe 3:
##### Berechnen Sie und lassen Sie ausgeben: Wie wahrscheinlich ist es, dass die nächste Bestellung eines Users 'organic bananas' enthält ...
##### a) ... wenn die vorherige Bestellung organic bananas enthalten hat.
##### b) ... wenn die vorherige Bestellung KEINE organic bananas enthalten hat.
##### c) ... ohne Unterscheidung nach dem Inhalt der vorherigen Bestellung.

Verwenden Sie als Schätzwert für die gesuchte Wahrscheinlichkeit die relative Häufigkeit aus den vorliegenden Daten. Es soll keine User-spezifische Schätzung erfolgen (dafür würde die Datenmenge auch nicht ausreichen), sondern für jede der Teilaufgaben soll EIN Wert über alle User hinweg ermittelt werden.  

In [ ]:
# Spalte 'previous_order_contains_bananas' zum orders DF hinzufügen:
h = orders[['order_id','user_id','order_number','contains_bananas']]
h.head()

,order_id,user_id,order_number,contains_bananas
0,2539329,1,1,0
1,2398795,1,2,1
2,473747,1,3,0
3,2254736,1,4,0
4,431534,1,5,1


In [ ]:
# copy complete h
h2 = h.copy()

In [ ]:
# set order number to previous order number (-1)
h2['order_number'] -=1
h2.head()

,order_id,user_id,order_number,contains_bananas
0,2539329,1,0,0
1,2398795,1,1,1
2,473747,1,2,0
3,2254736,1,3,0
4,431534,1,4,1


In [ ]:
# The columns named *_x are from the previous order, the ones from *_y from the current order
order_pairs = h.merge(h2, on=['user_id', 'order_number'])

In [ ]:
order_pairs.head(20)

,order_id_x,user_id,order_number,contains_bananas_x,order_id_y,contains_bananas_y
0,2539329,1,1,0,2398795,1
1,2398795,1,2,1,473747,0
2,473747,1,3,0,2254736,0
3,2254736,1,4,0,431534,1
4,431534,1,5,1,3367565,0
5,3367565,1,6,0,550135,0
6,550135,1,7,0,3108588,0
7,3108588,1,8,0,2295261,0
8,2295261,1,9,0,2550362,0
9,2550362,1,10,0,1187899,0


In [ ]:
b_b = len(order_pairs[(order_pairs['contains_bananas_x'] == 1 ) & (order_pairs['contains_bananas_y'] == 1 )].index)
b_b

202884

In [ ]:
nb_b = len(order_pairs[(order_pairs['contains_bananas_x'] == 0 ) & (order_pairs['contains_bananas_y'] == 1 )].index)
nb_b

175101

In [ ]:
b_nb = len(order_pairs[(order_pairs['contains_bananas_x'] == 1 ) & (order_pairs['contains_bananas_y'] == 0 )].index)
b_nb

194989

In [ ]:
nb_nb = len(order_pairs[(order_pairs['contains_bananas_x'] == 0 ) & (order_pairs['contains_bananas_y'] == 0 )].index)
nb_nb

2641900

In [ ]:
total_orders = b_b + nb_b + b_nb + nb_nb
total_orders

3214874

In [ ]:
# 3a:
b_and_b = b_b / (b_b + b_nb)

In [ ]:
# 3b 
b_no_b = nb_b / (nb_b + nb_nb)

In [ ]:
# 3c
b_total = (nb_b + b_b) / total_orders

In [ ]:
order_pairs.head(20)

,order_id_x,user_id,order_number,contains_bananas_x,order_id_y,contains_bananas_y
0,2539329,1,1,0,2398795,1
1,2398795,1,2,1,473747,0
2,473747,1,3,0,2254736,0
3,2254736,1,4,0,431534,1
4,431534,1,5,1,3367565,0
5,3367565,1,6,0,550135,0
6,550135,1,7,0,3108588,0
7,3108588,1,8,0,2295261,0
8,2295261,1,9,0,2550362,0
9,2550362,1,10,0,1187899,0


In [ ]:
#2. Einfaches Bananan Prognosemodell

In [ ]:
#def hasBananas(user_id):
#    users_orders = order_pairs.loc[order_pairs['user_id'] == user_id]
#    users_last_order_nr = users_orders.order_number.max()
#    users_last_order = users_orders.loc[users_orders['order_number']==users_last_order_nr].values
#   users_last_order = users_last_order[0][3]
    
    
   
#    return users_last_order
    

0


In [ ]:
def predict(user_id):
    bool = hasBananas(user_id)
    score = 0
    if bool:
        score += b_and_b
    else:
        score += b_no_b
    
    #score = 0.5 * score + 0.5 * b_total
        
    return round(score)

In [ ]:
prediction_bananas = [predict(i) for i in order_pairs['user_id']]

KeyboardInterrupt: ignored

In [ ]:
trueLabels = order_pairs['contains_bananas_y']

In [ ]:
prediction_all = [1 if i==1 else 0 for i in order_pairs['contains_bananas_x']]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(trueLabels, prediction_all))
print(precision_score(trueLabels, prediction_all))
print(recall_score(trueLabels, prediction_all,  average='macro'))
print(f1_score(trueLabels, prediction_all))

0.8848819580487447
0.509921507616752
0.7340090338472831
0.5229926094723518


In [ ]:
def cost(true, pred):
  costs = 0
  for t, p in zip(true, pred):
    
    if ((p==1) & (t==0)):
      costs += 0.11
    if ((p==0) & (t==1)):
      costs += 0.89
  return costs

print(cost(trueLabels, prediction_all))
print(cost(trueLabels, prediction_all)/(len(trueLabels)))

177288.6799999355
0.05514638520823382


In [ ]:
y_pred_ones = np.ones(len(trueLabels))
y_pred_zeros = np.zeros(len(trueLabels))


In [ ]:
print(accuracy_score(trueLabels, y_pred_ones))
print(precision_score(trueLabels, y_pred_ones))
print(recall_score(trueLabels, y_pred_ones,  average='macro'))
print(f1_score(trueLabels, y_pred_ones))
print(cost(trueLabels, y_pred_ones))
print(cost(trueLabels, y_pred_ones)/(len(trueLabels)))

0.11757381471248951
0.11757381471248951
0.5
0.21040903636908656
312057.78997765464
0.09706688037467553


In [ ]:
print(accuracy_score(trueLabels, y_pred_zeros))
print(precision_score(trueLabels, y_pred_zeros))
print(recall_score(trueLabels, y_pred_zeros,  average='macro'))
print(f1_score(trueLabels, y_pred_zeros))
print(cost(trueLabels, y_pred_zeros))
print(cost(trueLabels, y_pred_zeros)/(len(trueLabels)))

0.8824261852875105


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0
0.5
0.0
336406.650003144
0.10464069509509362
